In [7]:
use_most_recent = True
custom_filepath = ""

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import (
    NMF, 
    LatentDirichletAllocation,
    TruncatedSVD
)
import requests, gensim, spacy, json, nltk
nltk.download("stopwords")
import glob
# warnings.filterwarnings('ignore')
# from sklearn.model_selection import GridSearchCV
# from nltk.tokenize import wordpunct_tokenize
# from nltk.metrics import ConfusionMatrix
# from nltk.tokenize import word_tokenize
# from nltk.probability import FreqDist
# from nltk.stem import PorterStemmer, WordNetLemmatizer
# import matplotlib.pyplot as plt
# from functools import reduce
# from io import StringIO
# from time import time
import pandas as pd
# import numpy as np
# import re
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pandas as pd

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
# Get stopwords for Porguese and add prominent missing stopword (verb "to be")
# Full list of stopwords printed in appendix
pt_stopwords = nltk.corpus.stopwords.words('portuguese')
en_stopwords = nltk.corpus.stopwords.words('english')
html_elements = ['li', 'ul']
pt_stopwords.append("ser")
nltk_stopwords = pt_stopwords + en_stopwords + html_elements

In [10]:
if use_most_recent:
    filename = max(glob.iglob(f"data/csvfiles/*.csv"), key = os.path.getmtime)
    df = pd.read_csv(filename)
else:
    df = pd.read_csv(custom_filepath)
docs_raw = df['text'].astype(str)

In [11]:
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = nltk_stopwords,
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5, 
                                min_df = 10)
dtm_tf = tf_vectorizer.fit_transform(docs_raw)
tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())
dtm_tfidf = tfidf_vectorizer.fit_transform(docs_raw)

/mnt/c/Users/TFLORA/capstone-project/venv/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ate', 'eramos', 'estao', 'estavamos', 'estiveramos', 'estivessemos', 'foramos', 'fossemos', 'hao', 'houveramos', 'houverao', 'houveriamos', 'houvessemos', 'nao', 'sao', 'sera', 'serao', 'seriamos', 'tambem', 'tera', 'terao', 'teriamos', 'tinhamos', 'tiveramos', 'tivessemos', 'voce', 'voces'] not in stop_words.
  warnings.warn(
/mnt/c/Users/TFLORA/capstone-project/venv/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:2029: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. <class 'numpy.int64'> 'dtype' will be converted to np.float64.
  warnings.warn(


In [12]:
# for TF DTM
lda_tf = LatentDirichletAllocation(n_components=10, random_state=0)
lda_tf.fit(dtm_tf)
# for TFIDF DTM
lda_tfidf = LatentDirichletAllocation(n_components=10, random_state=0)
lda_tfidf.fit(dtm_tfidf)

LatentDirichletAllocation(random_state=0)

In [13]:
pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer)

/mnt/c/Users/TFLORA/capstone-project/venv/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/mnt/c/Users/TFLORA/capstone-project/venv/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/mnt/c/Users/TFLORA/capstone-project/venv/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/mnt/c/Users/TFLORA/capstone-project/venv/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's document

TypeError: Object of type complex is not JSON serializable

PreparedData(topic_coordinates=                        x                   y  topics  cluster       Freq
topic                                                                    
1     -0.167923+0.000000j  0.054880+0.000000j       1        1  43.046418
9      0.157644+0.000000j  0.083125+0.000000j       2        1  32.981128
6      0.023922+0.000000j -0.160773+0.000000j       3        1  16.194262
5     -0.000848+0.000000j  0.004531+0.000000j       4        1   1.446554
0     -0.002132+0.000000j  0.003040+0.000000j       5        1   1.055273
2     -0.002132+0.000000j  0.003040+0.000000j       6        1   1.055273
3     -0.002132+0.000000j  0.003040+0.000000j       7        1   1.055273
4     -0.002132+0.000000j  0.003040+0.000000j       8        1   1.055273
7     -0.002132+0.000000j  0.003040+0.000000j       9        1   1.055273
8     -0.002132+0.000000j  0.003040+0.000000j      10        1   1.055273, topic_info=            Term     Freq     Total Category  logprob  loglift
617   

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2392c230-4672-47ef-bbd2-e97d3feaff76' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>